### LSTM/GRU models 

Applying Deep Learning methods to predict one step-ahead daily energy household consumption. Data taken from [here](https://www.kaggle.com/datasets/jaganadhg/house-hold-energy-data/code). While notebook one implemented a [SARIMAX model](https://github.com/BorjaDaguerre/Predicting_promotions/blob/main/(1)%20Household_energy_forecasting_SARIMAX.ipynb), here the goal is to leverage the non-linearily capacities of DL models to improve predictions. Model parameters have been searched for using validation data and different hyperparamters permutations, the lagged values considered for this methods have also been informed by the results of finding the 'p' and 'q' values in notebook 1 and the acf and pacf graphs present there. ALthough the dataset is maybe not long enough to demostrate benefits of using a DL model to capture compex time series features, the time series does not present very defined patterns (seasonality, trend) and could benefit from using the non-linear methods deployed here.


In [1]:
# IDEAS:AUTOREGRESSIVE MODELS: comparing classification versus regression methods. predicting by day 
#(too short dataset ~720 datapoints)? 'reverse engineering' the cost of energy?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
import datetime as dt
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization
from keras.layers import Dense, Dropout 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Load data, drop useless columns  

df = pd.read_csv('D202.csv')
df = df.drop(columns = ['NOTES','TYPE', 'UNITS'] )

#number of years the time series covers
len(np.unique(df['DATE']))/365

2.0082191780821916

In [3]:
#Check the data types of the dataframe

df.dtypes

DATE           object
START TIME     object
END TIME       object
USAGE         float64
COST           object
dtype: object

In [5]:
# Create a column with the data and the time, to create daily and hourly datasets 

df['FULL_DATE'] = ''

for date in df['DATE'].items():
    df['FULL_DATE'].loc[date[0]] = date [1] + ' ' + df['START TIME'].loc[date[0]] 
    
df['FULL_DATE'] = pd.to_datetime(df['FULL_DATE'])
df.index = df['FULL_DATE']

In [6]:
# Create hourly and daily datasets for forecasting

df['HOUR'] = [dt.replace(minute=0, second=0) for dt in df['FULL_DATE']]
df['DATE'] = [date for date in df.index.date]

daily_usage = df.groupby(['DATE']).agg({'USAGE':'sum'})
hourly_usage = df.groupby(['HOUR']).agg({'USAGE':'sum'})

In [7]:
# Separate trainning, validation, and daily data test sets for hourly and daily data

train_daily = daily_usage[:int(len(daily_usage) * 0.5)]
train_hourly = hourly_usage[:int(len(hourly_usage) * 0.5)]
validation_daily = daily_usage[int(len(daily_usage)* 0.5):-int(len(daily_usage) * 0.25)]
validation_hourly = hourly_usage[int(len(hourly_usage)* 0.5):-int(len(hourly_usage) * 0.25)]
test_daily = daily_usage[-int(len(daily_usage) * 0.25):]
test_hourly = hourly_usage[-int(len(hourly_usage) * 0.25):]

In [8]:
#Dinamically forecasting since ARIMA default model 'cheats'



def one_step_ahead_DL(modelo, train_data, test_data, n_future, n_past, n_nodes_1, n_nodes_2, dropout, epochs, batch_size):
    """
    Perform one-step-ahead dynamic forecasting using a deep learning model.

    Parameters:
        modelo: The type of model architecture to use (e.g., LSTM).
        train_data (pd.DataFrame): Training data including both historical and validation data.
        test_data (pd.DataFrame): Testing data.
        n_future (int): Number of future steps to forecast.
        n_past (int): Number of past steps to use for prediction.
        n_nodes_1 (int): Number of nodes in the first LSTM layer.
        n_nodes_2 (int): Number of nodes in the second LSTM layer.
        dropout (float): Dropout rate.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.

    Returns:
        predictions (list): List of forecasted values.
    """

    predictions = []
    full_data = pd.concat([train_data, test_data], axis=0)
    train_X, train_y, test_X = [], [], []

    # Prepare training data
    for i in range(n_past, len(full_data) - n_future + 1):
        train_data_array = np.array(full_data)
        train_X.append(train_data_array[i - n_past:i, 0:train_data_array.shape[1]])
        train_y.append(train_data_array[i + n_future - 1:i + n_future, 0:train_data.shape[1]])

    # Prepare testing data
    for i in range(len(train_data), len(train_data) + len(test_data)):
        full_data_array = np.array(full_data)
        test_X.append(full_data_array[i - n_past:i, 0:test_data.shape[1]])

    train_X, train_y, test_X = np.array(train_X), np.array(train_y), np.array(test_X)

    # Initialize and compile the model
    model = Sequential()
    model.add(modelo(n_nodes_1, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
    model.add(modelo(n_nodes_2, activation='relu', return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(train_y.shape[1]))
    model.compile(optimizer='adam', loss='mse')

    # Train and forecast iteratively
    for i in range(len(train_data) - n_past, len(full_data) - n_past):
        model.fit(train_X[:i], train_y[:i], epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=False)
        prediction = model.predict(test_X[i - train_X.shape[0]].reshape(1, train_X.shape[1], train_X.shape[2]))
        predictions.append(prediction[0][0])

    return predictions

    

In [37]:
# Array creation for hyperarameter tunning and lags feed to the model

full_data = pd.concat([train_daily,validation_daily],axis = 0)
train_X, train_y, val_X, val_y = [], [], [], []

n_past, n_future = 7, 1
    
for i in range(n_past, len(train_daily) - n_future + 1):
    train_data = np.array(train_daily)
    train_X.append(train_data[i - n_past:i, 0:train_data.shape[1]])
    train_y.append(train_data[i + n_future-1:i+n_future,0:train_data.shape[1]])
    
for i in range(n_past, len(validation_daily) + - n_future + 1):
    val_data = np.array(validation_daily)
    val_X.append(val_data[i - n_past:i])
    val_y.append(val_data[i + n_future-1:i+n_future])


train_X, train_y, val_X, val_y = np.array(train_X), np.array(train_y), np.array(val_X), np.array(val_y)

In [38]:
# Function and nested loops for hyperparameter tunning forecasting the whole validation dataset. Hyperparameters consider
# where informed two factors: the short lenght of the train set (366 observations) and its univariety while also having in mind
# the irregular patterns present in them

def create_model(modelo,n_nodes_1, n_nodes_2, dropout):
    """
    Create a LSTM model with specified architecture.

    Parameters:
        n_nodes_1 (int): Number of nodes in the first GRU layer.
        n_nodes_2 (int): Number of nodes in the second GRU layer.
        dropout (float): Dropout rate.

    Returns:
        model (Sequential): Compiled Keras model.
    """

    model = Sequential()
    model.add(modelo(n_nodes_1, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
    model.add(BatchNormalization())
    model.add(modelo(n_nodes_2, activation='relu', return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(train_y.shape[1]))
    model.compile(optimizer='adam', loss='mse')
    
    return model


# Hyperparameters to tune
n_nodes_1_values = [8, 16, 32]
n_nodes_2_values = [8, 16, 32]
dropout_values = [0.2, 0.5, 0.7]
batch_size = [8, 16, 32]
best_model = None
best_nodes_1 = None
best_nodes_2 = None
best_dropout = None
best_batch_size = None

best_loss = float('inf')

for n_nodes_1 in n_nodes_1_values:
    for n_nodes_2 in n_nodes_2_values:
        for dropout in dropout_values:
            for batch in batch_size:
                model = create_model(GRU, n_nodes_1, n_nodes_2, dropout)
                history = model.fit(train_X, train_y, epochs=30, batch_size=batch, validation_data=(val_X, val_y), verbose=0)
                val_loss = history.history['val_loss'][-1]
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_nodes_1 = n_nodes_1
                    best_nodes_2 = n_nodes_2
                    best_dropout = dropout
                    best_batch_size = batch

NameError: name 'epoch' is not defined

In [9]:
# Adding the validation set to the train set for the final test

train_full = pd.concat([train_daily, validation_daily], axis = 0)

In [19]:
# Average over 10 iterations of the best validation LSTM results: #16 16 0.7 8 30 for 7 lagged values 

results = []

for i in range(0, 10):
    results_model = one_step_ahead_DL(LSTM, train_full, test_daily, 1, 7, 16 ,16, 0.7, 8, 30)
    results.append(results_model)
    


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


In [64]:
# Finding the average over the 10 predictions

average_predictions_LSTM = []

for idx in range(0, len(results[0])):
    average = []
    for lst in results:
        average.append(lst[idx])
    average_predictions_LSTM.append(np.mean(average))

average_predictions_LSTM = pd.DataFrame(average_predictions_LSTM, index = test_daily.index)

In [78]:
# MAE and predictions/real values plot for the LSTM model

round(mean_absolute_error(average_predictions_LSTM, test_daily),3)

plt.figure(figsize = (12,6), dpi = 1000)
plt.plot(average_predictions_LSTM, label = 'predicted')
plt.plot(test_daily, label = 'actual')
plt.title('One-step ahead LSTM forecast')
plt.legend()


In [ ]:
# Average over 10 iterations of the best validation GRU results: #16 16 0.7 8 30

results_GRU = []

for i in range(0, 10):
    m_results_GRU = one_step_ahead_DL(GRU, train_full, test_daily, 1, 7, 32 ,8, 0.2, 32, 30)
    results_GRU.append(m_results_GRU)

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 17ms/step


In [ ]:
# Finding the average over the 10 predictions

average_predictions_GRU = []

for idx in range(0, len(results[0])):
    average = []
    for lst in results_GRU:
        average.append(lst[idx])
    average_predictions_GRU.append(np.mean(average))

average_predictions_GRU = pd.DataFrame(average_predictions_GRU, index = test_daily.index)

In [ ]:
# MAE

round(mean_absolute_error(average_predictions_GRU, test_daily),3)


In [24]:
# Predictions/real values plot

plt.figure(figsize = (12,6), dpi = 1000)
plt.plot(average_predictions_GRU, label = 'predicted')
plt.plot(test_daily, label = 'actual')
plt.title('One-step ahead GRU forecast')
plt.legend()


6.779